In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy import signal
from tqdm import tqdm
from shapely.wkt import loads

from shapely.geometry import Point

## On fusionne les données débit/météo

### Stations

In [4]:
stations_debit = pd.read_csv("../../Data/Extraction_Hydro/Stations.csv")
stations_debit.head()

,Code station,Libelle station,Regime influencé,Altitude du zéro de l'échelle,geometry,Ordre,Cours eau
0,O0200020,La Garonne [partielle] à Saint-Gaudens [Valent...,pas ou faiblement,357.00,POINT (0.7068033357391884 43.09775183863328),0,Garonne
1,O2620010,La Garonne à Verdun-sur-Garonne,pas ou faiblement,90.00,POINT (1.242176195808334 43.85463266047086),1,Garonne
2,O6140010,La Garonne à Lamagistère,pas ou faiblement,46.00,POINT (0.8312973485763152 44.12107242091946),2,Garonne
3,O9000010,La Garonne à Tonneins,pas ou faiblement,0.00,POINT (0.2221525107266094 44.41192952559171),3,Garonne
4,K0030020,La Loire à Issarlès [Pont de La Borie],fortement,881.34,POINT (4.048486196061388 44.81973247281849),0,Loire


In [5]:
stations_debit.to_csv("../../Data/Base/Stations_Debit.csv",
                     index = False)

In [9]:
stations_meteo = pd.read_csv("../../Data/Extraction_Meteo/Stations.csv")
stations_meteo

,ID,Nom,Latitude,Longitude,Altitude,geometry
0,7005,ABBEVILLE,50.136000,1.834000,69,POINT (1.834 50.136)
1,7015,LILLE-LESQUIN,50.570000,3.097500,47,POINT (3.0975 50.57)
2,7027,CAEN-CARPIQUET,49.180000,-0.456167,67,POINT (-0.456167 49.18)
3,7037,ROUEN-BOOS,49.383000,1.181667,151,POINT (1.181667 49.383)
4,7072,REIMS-PRUNAY,49.209667,4.155333,95,POINT (4.155333 49.209667)
5,7110,BREST-GUIPAVAS,48.444167,-4.412000,94,POINT (-4.412 48.444167)
6,7117,PLOUMANAC'H,48.825833,-3.473167,55,POINT (-3.473167 48.825833)
7,7130,RENNES-ST JACQUES,48.068833,-1.734000,36,POINT (-1.734 48.068833)
8,7139,ALENCON,48.445500,0.110167,143,POINT (0.110167 48.4455)
9,7149,ORLY,48.716833,2.384333,89,POINT (2.384333 48.716833)


In [10]:
stations_meteo.to_csv("../../Data/Base/Stations_Meteo.csv",
                     index=False)

### Mesures

In [15]:
mesures_debit = pd.read_csv("../../Data/Extraction_Hydro/Mesures.csv")
mesures_debit["Date"] = pd.to_datetime(mesures_debit["Date"], format = "%Y/%m/%d %H:%M:%S")
mesures_debit.head()

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,H0100020,H0400010,H0400020,H0800011,H0800012,H0810010,H1700010,H3930020,H4340020,H8100021
0,2010-01-01 00:00:00,41.3,122.0,285.0,506.0,0.42,2.87,9.33,22.8,6.90,...,25.0,59.9,9.39,25.3,13.7,26.7,55.8,308.0,306.0,703.0
1,2010-01-01 02:00:00,47.4,125.0,291.0,520.0,0.44,2.86,9.93,21.6,7.01,...,25.1,60.1,9.50,25.0,13.5,27.5,56.6,308.0,303.0,710.0
2,2010-01-01 04:00:00,49.2,132.0,300.0,516.0,0.50,2.84,10.80,21.9,7.03,...,25.2,60.1,9.60,24.7,13.4,28.3,57.4,310.0,302.0,716.0
3,2010-01-01 06:00:00,42.4,135.0,314.0,496.0,0.54,2.90,10.20,20.8,7.04,...,25.3,60.1,9.63,24.4,13.3,29.2,58.1,312.0,301.0,724.0
4,2010-01-01 08:00:00,47.3,132.0,329.0,472.0,0.54,3.04,10.70,21.4,7.12,...,25.3,60.1,9.64,24.2,13.2,29.9,58.7,313.0,300.0,742.0


In [19]:
liste_stations = list(stations_debit["Code station"])

mesures_moyennes = pd.DataFrame()
liste_annees = np.unique(mesures_debit["Date"].apply(lambda x: x.year))
for curr_annee in liste_annees:
    mesures_annee = mesures_debit[mesures_debit["Date"].apply(lambda x: x.year == curr_annee)]
    for curr_month in range(1, 13):
        mesures_mois = mesures_annee[mesures_annee["Date"].apply(lambda x: x.month == curr_month)]
        mesures_mois = mesures_mois.groupby(mesures_mois["Date"].apply(lambda x: x.day)).mean()
        mesures_mois["Jour"] = mesures_mois.index 
        mesures_mois["Date"] = pd.to_datetime(str(curr_annee) + "/" + str(curr_month).rjust(2, "0") + "/" + mesures_mois["Jour"].apply(lambda x: str(x).rjust(2, "0")))
        mesures_moyennes = pd.concat([mesures_moyennes, mesures_mois])
colonnes = ["Date"] + liste_stations
mesures_debit = mesures_moyennes[colonnes]
mesures_debit

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,H0100020,H0400010,H0400020,H0800011,H0800012,H0810010,H1700010,H3930020,H4340020,H8100021
Date,,,,,,,,,,,,,,,,,,,,,
1,2010-01-01,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,...,25.425000,60.041667,9.601667,24.050000,13.166667,30.041667,59.700000,317.333333,302.666667,730.166667
2,2010-01-02,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,...,25.116667,57.666667,9.186667,22.250000,12.458333,31.425000,66.425000,345.500000,331.250000,736.833333
3,2010-01-03,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,...,22.958333,53.583333,9.015833,21.041667,12.025000,30.691667,68.141667,306.916667,314.666667,783.083333
4,2010-01-04,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,...,20.741667,47.783333,9.050833,20.616667,11.833333,30.175000,65.316667,269.916667,261.000000,687.333333
5,2010-01-05,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,...,19.133333,43.241667,9.143333,19.225000,13.016667,29.958333,62.350000,245.750000,239.750000,691.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,2020-12-27,41.291667,188.250000,378.833333,627.666667,0.672500,3.170000,13.083333,30.383333,69.258333,...,16.725000,41.558333,15.008333,8.342500,15.566667,21.225000,47.258333,226.666667,256.500000,595.083333
28,2020-12-28,51.316667,243.500000,425.666667,676.250000,0.683333,3.257500,13.516667,32.983333,85.591667,...,18.666667,45.491667,15.666667,9.336667,17.275000,22.583333,48.200000,248.000000,302.250000,711.666667
29,2020-12-29,47.250000,552.666667,1474.750000,1975.833333,0.702500,3.126667,13.058333,35.491667,83.050000,...,20.391667,50.633333,24.841667,12.816667,20.991667,26.108333,50.350000,308.833333,353.666667,846.833333


In [21]:
mesures_meteo = pd.read_csv("../../Data/Extraction_Meteo/Mesures.csv")
mesures_meteo["Date"] = pd.to_datetime(mesures_meteo["Date"], format = "%Y/%m/%d %H:%M:%S")
mesures_meteo.head()

,Date,Pression_07005,Vent_Nord_07005,Vent_Est_07005,Vitesse_vent_07005,Temperature_07005,Humidite_07005,Precipitations_07005,Pression_07015,Vent_Nord_07015,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01 01:00:00,98810.0,5.826094,2.120525e+00,6.2,0.5,83.0,0.0,99120.0,4.366453,...,10.5,76.0,0.2,99500.0,-7.235633,-2.633555,7.7,14.2,90.0,0.0
1,2010-01-01 04:00:00,98800.0,6.200000,-1.518562e-15,6.2,0.2,82.0,0.0,99110.0,4.416730,...,10.0,61.0,0.0,99180.0,-7.507236,-6.299319,9.8,14.3,90.0,0.0
2,2010-01-01 07:00:00,98910.0,5.022520,8.856057e-01,5.1,0.2,80.0,0.0,99180.0,5.802370,...,10.8,45.0,0.0,99200.0,-1.744303,-4.792432,5.1,12.2,72.0,0.8
3,2010-01-01 10:00:00,99120.0,5.700000,-1.396097e-15,5.7,0.2,81.0,0.0,99360.0,6.235383,...,12.0,39.0,0.0,99360.0,-2.952019,-16.741732,17.0,11.0,80.0,0.4
4,2010-01-01 13:00:00,99300.0,5.022520,8.856057e-01,5.1,1.7,64.0,0.0,99550.0,4.749476,...,12.0,50.0,0.0,99370.0,-2.500534,-14.181232,14.4,12.3,69.0,0.0


In [24]:
liste_stations = list(stations_meteo["ID"].apply(lambda x: str(x).rjust(5, "0")))
colonnes_meteo = ["Pression", "Vent_Nord", "Vent_Est", "Vitesse_vent", "Temperature", "Humidite", "Precipitations"]

mesures_moyennes = pd.DataFrame()
liste_annees = np.unique(mesures_meteo["Date"].apply(lambda x: x.year))
for curr_annee in liste_annees:
    mesures_annee = mesures_meteo[mesures_meteo["Date"].apply(lambda x: x.year == curr_annee)]
    for curr_month in range(1, 13):
        mesures_mois = mesures_annee[mesures_annee["Date"].apply(lambda x: x.month == curr_month)]
        mesures_mois = mesures_mois.groupby(mesures_mois["Date"].apply(lambda x: x.day)).mean()
        mesures_mois["Jour"] = mesures_mois.index 
        mesures_mois["Date"] = pd.to_datetime(str(curr_annee) + "/" + str(curr_month).rjust(2, "0") + "/" + mesures_mois["Jour"].apply(lambda x: str(x).rjust(2, "0")))
        mesures_moyennes = pd.concat([mesures_moyennes, mesures_mois])
colonnes = ["Date"]
for code in liste_stations:
    colonnes += [tpe + "_" + code for tpe in colonnes_meteo]
mesures_meteo = mesures_moyennes[colonnes]
mesures_meteo

,Date,Pression_07005,Vent_Nord_07005,Vent_Est_07005,Vitesse_vent_07005,Temperature_07005,Humidite_07005,Precipitations_07005,Pression_07015,Vent_Nord_07015,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
Date,,,,,,,,,,,,,,,,,,,,,
1,2010-01-01,99303.75,4.157911,-0.221807,4.6375,0.5875,80.625,0.0000,99572.50,4.364498,...,10.8375,54.500,0.025,99478.75,-2.842313,-10.281279,11.5000,12.4250,78.125,0.5500
2,2010-01-02,100673.75,-2.595459,-1.592976,3.2875,0.1250,91.750,0.1250,100967.50,-1.763227,...,5.2250,62.500,0.000,100988.75,0.439673,-5.555278,6.0375,11.0000,56.875,0.0000
3,2010-01-03,101200.00,0.620105,1.030131,4.2375,-1.5000,82.500,0.0000,101555.00,1.447183,...,4.9250,66.875,0.000,101792.50,1.802059,1.524495,3.3375,6.5750,64.500,0.0000
4,2010-01-04,100968.75,-0.676317,0.798484,2.3875,-5.2625,85.375,0.0000,101288.75,-1.064316,...,5.4000,83.750,0.025,101070.00,1.809303,1.632121,2.4625,8.9750,85.625,0.0750
5,2010-01-05,99818.75,-2.505195,1.421730,3.1000,-3.8375,87.375,0.0000,100163.75,-1.607421,...,6.7750,86.875,0.325,99836.25,0.903730,1.437407,2.1750,10.8500,93.125,1.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,2020-12-27,97621.25,-3.929064,-1.356069,4.2125,6.1125,90.375,0.6125,97947.50,-8.941180,...,4.0625,58.375,0.000,100906.25,1.687684,0.985724,2.8000,5.9500,69.750,-0.0125
28,2020-12-28,96651.25,-1.309499,1.027574,2.5500,3.4875,92.875,-0.0250,97017.50,-2.576697,...,6.7000,60.875,0.000,99291.25,-3.083568,-4.526198,7.4750,11.5125,77.750,1.0375
29,2020-12-29,98015.00,-0.129731,-0.824674,1.7250,3.4375,94.875,0.0625,98281.25,-1.361862,...,8.4625,54.625,0.000,99955.00,2.336237,1.208835,2.7000,8.4250,89.250,0.3125


In [33]:
mesures_debit.index.name = "Index"
mesures_meteo.index.name = "Index"

In [34]:
mesures = mesures_debit.merge(mesures_meteo,
                             on = "Date",
                             how = "outer")

In [36]:
mesures = mesures.sort_values(by = "Date")
mesures

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,...,10.8375,54.500,0.025,99478.75,-2.842313,-10.281279,11.5000,12.4250,78.125,0.5500
1,2010-01-02,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,...,5.2250,62.500,0.000,100988.75,0.439673,-5.555278,6.0375,11.0000,56.875,0.0000
2,2010-01-03,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,...,4.9250,66.875,0.000,101792.50,1.802059,1.524495,3.3375,6.5750,64.500,0.0000
3,2010-01-04,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,...,5.4000,83.750,0.025,101070.00,1.809303,1.632121,2.4625,8.9750,85.625,0.0750
4,2010-01-05,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,...,6.7750,86.875,0.325,99836.25,0.903730,1.437407,2.1750,10.8500,93.125,1.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4013,2020-12-27,41.291667,188.250000,378.833333,627.666667,0.672500,3.170000,13.083333,30.383333,69.258333,...,4.0625,58.375,0.000,100906.25,1.687684,0.985724,2.8000,5.9500,69.750,-0.0125
4014,2020-12-28,51.316667,243.500000,425.666667,676.250000,0.683333,3.257500,13.516667,32.983333,85.591667,...,6.7000,60.875,0.000,99291.25,-3.083568,-4.526198,7.4750,11.5125,77.750,1.0375
4015,2020-12-29,47.250000,552.666667,1474.750000,1975.833333,0.702500,3.126667,13.058333,35.491667,83.050000,...,8.4625,54.625,0.000,99955.00,2.336237,1.208835,2.7000,8.4250,89.250,0.3125
4016,2020-12-30,47.225000,359.583333,1192.916667,2684.166667,0.760833,3.132500,12.375000,32.058333,43.616667,...,7.6375,57.375,0.000,100448.75,1.686884,1.855508,2.7750,7.1750,87.375,0.3125


In [37]:
mesures.to_csv("../../Data/Base/Mesures.csv",
              index=False)